In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [8]:
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.7)
output = llm("I want to open an Italian food restuarant, suggest fancy names for this.")
print(output)




1. "Cibo Italiano"
2. "Bella Italia"
3. "La Dolce Vita"
4. "Trattoria Elegante"
5. "Ristorante Di Lusso"
6. "Cucina Classica"
7. "Al Dente"
8. "Fresco Italiano"
9. "Gusto Italiano"
10. "Vivace Ristorante"
11. "Sapori d'Italia"
12. "Ristorante di Roma"
13. "Grazie Italiano"
14. "Sapore di Toscana"
15. "Bistro Fiore"
16. "Bella Tavola"
17. "La Cucina Bella"
18. "Ristorante Amore"
19. "Bella Cucina"
20. "Cucina di Lusso"


In [9]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open an {cuisine} food restuarant, suggest a fancy name for this."
)

prompt_template_name.format(cuisine="Mexican")

'I want to open an Mexican food restuarant, suggest a fancy name for this.'

In [11]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Gujarati")

'\n\n"Kathiyawadi Delights"'